# Evaluation
### 1. Export model inference graph 

In [1]:
exporter_main_v2_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/models/research/object_detection/exporter_main_v2.py'
trained_checkpoint_dir_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/sliced_images_demo/models/my_efficientdet_d4_coco17_tpu-32/v2/'
pipeline_config_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/sliced_images_demo/models/my_efficientdet_d4_coco17_tpu-32/v2/pipeline.config'
output_directory_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/sliced_images_demo/inference_graph/my_efficientdet_d4_coco17_tpu-32/v2/'

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"


In [ ]:
!python {exporter_main_v2_path} \
    --trained_checkpoint_dir {trained_checkpoint_dir_path} \
    --output_directory {output_directory_path} \
    --pipeline_config_path {pipeline_config_path}

## 2. Test trained model on test images

In [3]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [4]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: a file path (this can be local or on colossus)

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [5]:
labelmap_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  8


In [7]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

In [8]:
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2022-05-25 13:00:34.815096: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-25 13:00:39.853367: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9583 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1c:00.0, compute capability: 7.5
2022-05-25 13:00:39.855133: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9646 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1d:00.0, compute capability: 7.5
2022-05-25 13:00:39.856707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/repli

In [9]:
tf.keras.backend.clear_session()

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
model = tf.saved_model.load(f'{output_directory_path}/saved_model')

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 38.47862386703491 seconds


In [10]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [13]:

## Out-of-Sample Pictures
#test_images_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/data_raw/extras/*jpg'


## Eval-Pictures
test_images_path = '/home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/sliced_images_demo/images/test/*jpg'


# specify slice width=height
slice_size = 1280


with strategy.scope():
    for image_path in glob.glob(test_images_path):
        im = Image.open(image_path)
        imr = np.array(im, dtype=np.uint8)
        height = imr.shape[0]
        width = imr.shape[1]

        for i in range((height // slice_size)):
            for j in range((width // slice_size)):
                
                sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                #sliced_im = Image.fromarray(sliced)

                filename = image_path.split('/')[-1]
                slice_path = filename.replace('.jpg', f'_{i}_{j}.jpg')
                print(slice_path)
                
                image_np = sliced
                output_dict = run_inference_for_single_image(model, image_np)
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    output_dict['detection_boxes'],
                    output_dict['detection_classes'],
                    output_dict['detection_scores'],
                    category_index,
                    instance_masks=output_dict.get('detection_masks_reframed', None),
                    use_normalized_coordinates=True,
                    line_thickness=3)
                print(output_dict.keys())
                #display(Image.fromarray(image_np))

12210ad7-83f8-4b54-bb4b-e93f8ff6ac1f_1_1_0_0.jpg
dict_keys(['detection_scores', 'raw_detection_scores', 'detection_anchor_indices', 'raw_detection_boxes', 'detection_classes', 'detection_multiclass_scores', 'detection_boxes', 'num_detections'])
d3d2b706-9017-41f4-b57e-469038daa634_1_1_0_0.jpg
dict_keys(['detection_scores', 'raw_detection_scores', 'detection_anchor_indices', 'raw_detection_boxes', 'detection_classes', 'detection_multiclass_scores', 'detection_boxes', 'num_detections'])
014de911-7810-4f7d-8967-3e5402209f4a_1_1_0_0.jpg
dict_keys(['detection_scores', 'raw_detection_scores', 'detection_anchor_indices', 'raw_detection_boxes', 'detection_classes', 'detection_multiclass_scores', 'detection_boxes', 'num_detections'])
dacabde6-c8a7-475b-ab26-81b4e4dd5977_1_0_0_0.jpg
dict_keys(['detection_scores', 'raw_detection_scores', 'detection_anchor_indices', 'raw_detection_boxes', 'detection_classes', 'detection_multiclass_scores', 'detection_boxes', 'num_detections'])
ca7b1077-0d37-4ada-9

KeyboardInterrupt: 

In [ ]:
## Remove current session / unload current loaded model

tf.keras.backend.clear_session()